In [1]:
# Install liblinear: sudo apt-get install python-liblinear
# from liblinearutil import *
import numpy as np

In [2]:
def read_problem(file_name):
    y = []
    x = []
    feature_len = 22
    for line in open(file_name):
        tmp = line.split(' ',1)
        y = y + [int(tmp[0])]
        vec = np.zeros(feature_len)
        for each in tmp[1].split():
            index,val = each.split(':')
            vec[int(index)-1] = float(val)
        x.append(vec)
    y = np.array(y)
    x = np.array(x)
    return y,x

# Gradient ascent step for dual variables

In [16]:
def get_tp_tn(x,y,w):
    h = x.dot(w)
    ind = np.where(h>0)
    y1 = -1*np.ones_like(y)
    y1[ind] = 1
    tp = sum(np.multiply(1+y,1+y1)/4)
    tn = sum(np.multiply(1-y,1-y1)/4)
    return tp,tn

def get_projection(alpha,beta):
    if alpha>0 and beta>0 and alpha*beta>=1.0/4:
        return alpha,beta
    a = 16
    b = -16*beta
    c = 0
    d = 8*alpha #4
    e = -1
    D0 = c*c - 3*b*d + 12*a*e
    D1 = 2*c*c*c-9*b*c*d+27*b*b*e+27*a*d*d-72*a*c*e
    p = (8*a*c-3*b*b)/(8*a*a)
    q = (b*b*b-4*a*b*c+8*a*a*d)
    Q = np.power((D1+np.sqrt(D1*D1-4*D0*D0*D0))/2.0,1/3)
    s = 0.5*np.sqrt(-2/3*p+1/(3*a)*(Q+D0/Q))
    y1 = -float(b)/(4*a) - s + 0.5*np.sqrt(-4*s*s-2*p+q/s)
    y2 = -float(b)/(4*a) - s - 0.5*np.sqrt(-4*s*s-2*p+q/s)
    y3 = -float(b)/(4*a) + s + 0.5*np.sqrt(-4*s*s-2*p+q/s)
    y4 = -float(b)/(4*a) + s - 0.5*np.sqrt(-4*s*s-2*p+q/s)
    x1 = 1.0/(4*y1)
    x2 = 1.0/(4*y2)
    x3 = 1.0/(4*y3)
    x4 = 1.0/(4*y4)
    if x1>0 and y1>0:
        return x1,y1
    if x2>0 and y2>0:
        return x2,y2
    if x3>0 and y3>0:
        return x3,y3    
    if x4>0 and y4>0:
        return x4,y4
    return 1,1

# Change it to gradient descent
def gradient_ascent(x,y,w,a,b,alpha=0.1,maxiter=50):
    n = x.shape[0]
    n_pos = len([i for i in range(n) if y[i]==1])
    n_neg = n-n_pos
    ind_pos = [i for i in range(n) if y[i]>0]
    ind_neg = [i for i in range(n) if y[i]<0]
    x_pos = x[ind_pos]
    x_neg = x[ind_neg]
    tpr,tnr = get_tp_tn(x,y,w)
    tpr /= float(n_pos)
    tnr /= float(n_neg)
    epsilon = 0.0001
    print('TPR =',tpr,'TNR =',tnr)
    for it in range(maxiter):
#         alpha = 1.0/(it+1)
        # for positives
        tmp = 1 - (2*n_pos)/(a*n)*x_pos.dot(w)
        j = np.where(tmp>0)[0]
        tmp = np.zeros_like(tmp)
        tmp[j]=1.0/n_pos
        ga = sum(tmp)
        a -= alpha*ga
#         print('Missed pos:',len(j))
        # for negatives
        tmp = 1 + (2*n_neg)/(b*n)*x_neg.dot(w)
        j = np.where(tmp>0)[0]
        tmp = np.zeros_like(tmp)
        tmp[j]=1.0/n_neg
        gb = sum(tmp)
        b -= alpha*gb
#         print('Missed neg:',len(j))
        # Projection to a,b \in R+ \intersect ab>1/4
        print('Before Projection -> a = {} b= {}'.format(a,b))
        a,b = get_projection(a,b)
        print('After Projection  -> a = {} b= {}'.format(a,b))
#         print('ga =',ga,'gb =',gb)
    return a,b

## Liblinear for primal variable w

In [6]:
# customized liblinear
import sys
sys.path.insert(0, "/home/debojyoti/opt/liblinear-2.1")
from ppython import liblinear
from ppython.liblinear import *
from ppython.liblinearutil import *

In [17]:
def modifyx(x,y,a,b,n,n_pos,n_neg):
    scale_pos = float(2*n_pos)/(a*n)
    scale_neg = float(2*n_neg)/(b*n)
    for i in range(len(y)):
        if y[i]==1:
            x[i].update((key,val*scale_pos) for key,val in x[i].items())
        else:
            x[i].update((key,val*scale_neg) for key,val in x[i].items())
    return x
# init section
a,b = 0.5,0.5
y_lst,x_lst = read_problem('./data/ijcnn1.tr') # x as numpy list
n_dim = len(x_lst[0])
w = np.random.uniform(low=-1,high=1,size=(n_dim,))
# w = np.zeros(n_dim)
y_orig,x_orig = svm_read_problem('./data/ijcnn1.tr') # x in liblinear compatible dictionary format
n = len(y_lst)
n_pos = len([i for i in range(n) if y_lst[i]==1])
n_neg = n-n_pos
c = 5.0
# Iterative section: Gradient Ascent & Liblinear
for i in range(10):
    a,b = gradient_ascent(x_lst,y_lst,w,a,b) #Gradient ascent
    c1 = float(a)/n_pos
    c2 = float(b)/n_neg
    param = parameter('-s 3 -w1 {} -w-1 {} -c {}'.format(c1,c2,c))
    x = modifyx(x_orig,y_orig,a,b,n,n_pos,n_neg)
    prob = problem(y_orig,x)
    model = train(prob,param)
    # Get model parameters
    w = model.get_decfun()[0]
    print('----------Cycle End:  a =',a,'b =',b,'-------------')

TPR = 0.302781844802 TNR = 0.699350957733
Before Projection -> a = 0.40000000000000385 b= 0.46792781383567833
After Projection  -> a = 0.003031830038003088 b= 82.45844815386229
Before Projection -> a = -0.06674855063549402 b= 82.35844815386224
After Projection  -> a = 1 b= 1
Before Projection -> a = 0.9000000000000039 b= 0.9474972296976247
After Projection  -> a = 0.9000000000000039 b= 0.9474972296976247
Before Projection -> a = 0.8000000000000078 b= 0.8977900902326741
After Projection  -> a = 0.8000000000000078 b= 0.8977900902326741
Before Projection -> a = 0.7000000000000117 b= 0.8505968022795187
After Projection  -> a = 0.7000000000000117 b= 0.8505968022795187
Before Projection -> a = 0.6000000000000156 b= 0.8057369004273622
After Projection  -> a = 0.6000000000000156 b= 0.8057369004273622
Before Projection -> a = 0.5000000000000195 b= 0.763492164001834
After Projection  -> a = 0.5000000000000195 b= 0.763492164001834
Before Projection -> a = 0.4000000000000234 b= 0.7233211967705448


/home/debojyoti/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:24: RuntimeWarning: invalid value encountered in sqrt
/home/debojyoti/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:25: RuntimeWarning: invalid value encountered in sqrt
/home/debojyoti/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:26: RuntimeWarning: invalid value encountered in sqrt
/home/debojyoti/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:27: RuntimeWarning: invalid value encountered in sqrt


----------Cycle End:  a = 1 b = 1 -------------
TPR = 0.0 TNR = 1.0
Before Projection -> a = 0.9000000000000039 b= 0.9452018363146883
After Projection  -> a = 0.9000000000000039 b= 0.9452018363146883
Before Projection -> a = 0.8000000000000078 b= 0.9176191230014065
After Projection  -> a = 0.8000000000000078 b= 0.9176191230014065
Before Projection -> a = 0.7000000000000117 b= 0.9129871774576357
After Projection  -> a = 0.7000000000000117 b= 0.9129871774576357
Before Projection -> a = 0.6000000000000156 b= 0.9094095298400957
After Projection  -> a = 0.6000000000000156 b= 0.9094095298400957
Before Projection -> a = 0.5000000000000195 b= 0.9064302675320381
After Projection  -> a = 0.5000000000000195 b= 0.9064302675320381
Before Projection -> a = 0.4000000000000234 b= 0.9037391166692869
After Projection  -> a = 0.4000000000000234 b= 0.9037391166692869
Before Projection -> a = 0.30000000000002724 b= 0.9013297451321648
After Projection  -> a = 0.30000000000002724 b= 0.9013297451321648
Before